In [69]:
import xml.etree.cElementTree as ET
import os
import re
import pandas as pd
import json

# Supporting functions

In [9]:
def processing_1(form):
    form=form.replace('#', '').replace('[', '').replace(']', '').replace('<', '').replace('>', '').replace( '!', '').replace('?', '').replace('@c','').replace('@t','').replace('_','').replace(',','')
    #x = re.sub(r"\[\.+\]","unk",text_line)
    #x = re.sub(r"...","unk",x)
    '''x = re.sub(r'\#', '', text_line)
    x = re.sub(r"\_", "", x)
    x = re.sub(r"\[", "", x)
    x = re.sub(r"\]", "", x)
    x = re.sub(r"\<", "", x)
    x = re.sub(r"\>", "", x)
    x = re.sub(r"\!", "", x)
    x = re.sub(r"@c", "", x)
    x = re.sub(r"@t", "", x)
    x = re.sub(r",", "", x)
    #x=re.sub(r"(x)+","x",x)
    x = re.sub(r"\?", "", x)
    x = x.split()
    x = " ".join(x)
    k = re.search(r"[a-wyzA-Z]+",x)
    if k:
        return x
    else:
        return "" '''
    if(len(form)==0):
        form="..." 
    return form

In [70]:
def txt_file_read(filename):
    lines=[]
    with open(filename, "r") as f:
        for line in f:
            line=line.strip()
            lines.append(line)

    return lines

In [71]:
def json_file_read(filename):
    with open(filename) as f:
        data = json.load(f)
    return data

In [31]:
def process_row(row):
    orignal = row[1]
    segm = row[2]
    lem = "_"
    eng_translation = "_"
    # match pattern - all groups starting with word and containing paranthesese
    w_group = re.findall(r"[\(\)a-zA-Z0-9_-]+\[\w+\]",segm)
    for w in w_group:
        start_index = w.find('[')
        end_index = w.find(']')
        lem = w[:start_index]
        eng_translation = w[start_index+1:end_index]
        break
    tags = row[3].split('.')
    return orignal, lem,eng_translation,tags

## Gold corpus annotation 

In [40]:
# define dataframe
column = ['id','form','lem','eng_translation','tags']
extracted_df = pd.DataFrame(columns = column)
extracted_df.head()

,id,form,lem,eng_translation,tags


In [41]:
base_dir = '../Data/mtaac_gold_corpus/morph/to_dict/'
conll_files = os.listdir('../Data/mtaac_gold_corpus/morph/to_dict/')

In [49]:
for conll_file in conll_files:
    temp = os.path.join(base_dir,conll_file)
    f = txt_file_read(temp)
    f_lines = [line.split("\t") for line in f][2:]

    # start processing file
    processed_ls = []
    for row in f_lines:
        if len(row) > 3:
            orignal, lem, eng_translation,tags = process_row(row)
            # print(orignal, lem,eng_translation,tags)
            if orignal!="_" and lem!="_":
                processed_ls.append([row[0],orignal, lem,eng_translation,tags])
        
    processed_df = pd.DataFrame(processed_ls, columns=column)
    print(processed_df)
    extracted_df = extracted_df.append(processed_df)

        id           form       lem eng_translation              tags
0    o.1.1        1(disz)   1(disz)             one              [NU]
1    o.1.2      udu-nita2   udunita             ram          [N, ABS]
2    o.1.3        ba-usz2    ba-usz             die  [MID, V, 3-SG-S]
3    o.2.1        1(disz)   1(disz)             one              [NU]
4    o.2.2          sila4      sila            lamb               [N]
5    o.2.3        ASZ-ur4      azur   type_of_sheep               [N]
6    o.2.4        ba-usz2    ba-usz             die  [MID, V, 3-SG-S]
7    o.3.1             ki        ki           place               [N]
8    o.3.2       ur-u3-ta       Uru               1    [PN, GEN, ABL]
9    o.4.1           sza3      szag           heart               [N]
10   o.4.2         bala-a      bala             tax      [N, GEN, L1]
11   r.1.1        kiszib3    kiszib            seal               [N]
12   r.1.2          ARAD2      Arad               1         [PN, GEN]
13   r.2.1          

/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_72273/4058718997.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_72273/4058718997.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_72273/4058718997.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_72273/4058718997.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

        id             form        lem eng_translation  \
0    o.1.1          7(disz)    7(disz)             one   
1    o.1.2          {gi}hal        hal          basket   
2    o.1.3            esir2       esir         bitumen   
3    o.1.4            su-ba        sub             rub   
4    o.2.1               ki         ki           place   
5    o.2.2  szesz-kal-la-ta  Szeszkala               1   
6    o.4.1              szu        szu            hand   
7    o.4.2            ba-ti       -teg          accept   
8    r.1.1              iti        iti           month   
9    r.1.2         min-esz3     Minesz               1   
10   r.2.1               mu         mu            year   
11   r.2.2       an-sza{ki}     Anszan               1   
12   r.2.3           ba-hul    ba-hulu         destroy   
13  s1.1.1    ur-{d}li9-si4     Urlisi               1   
14  s1.2.1            ensi2      ensik           ruler   
15  s1.2.2         umma{ki}       Umma               1   
16  s1.3.1    

/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_72273/4058718997.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_72273/4058718997.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_72273/4058718997.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_72273/4058718997.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas


      id         form      lem eng_translation                     tags
0  o.1.1   {d}szul-gi  Szulgir               1                     [RN]
1  o.2.1         nita     nita            male                      [N]
2  o.2.2       kal-ga    kalag          strong              [NF, V, PT]
3  o.3.1        lugal    lugal            king                      [N]
4  o.3.2  uri5{ki}-ma     Urim               1                [SN, GEN]
5  o.4.1        lugal    lugal            king                      [N]
6  o.4.2     an-ub-da   anubda         quarter                      [N]
7  o.4.3    limmu2-ba    limmu            four  [NU, 3-SG-NH-POSS, GEN]
       id                form      lem      eng_translation              tags
0   o.1.1             1(disz)  1(disz)                  one              [NU]
1   o.1.2                 dug      dug                  pot               [N]
2   o.1.3                dida     dida           sweet_wort               [N]
3   o.1.4                  du      gin 

...

/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_72273/4058718997.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_72273/4058718997.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_72273/4058718997.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_72273/4058718997.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

        id                    form            lem eng_translation  \
0    o.1.1                 8(disz)        8(disz)             one   
1    o.1.2                    kusz           kusz         leather   
2    o.1.3                     gu4            gud              ox   
3    o.2.1                3(gesz2)        3(gesz)           sixty   
4    o.2.2                 5(disz)        5(disz)             one   
5    o.2.3                    kusz           kusz         leather   
6    o.2.4                     udu            udu           sheep   
7    o.3.1                      ki             ki           place   
8    o.3.2         ur-{d}ha-ia3-ta             ia               1   
9    o.4.1                    kusz           kusz         leather   
10   o.4.2       {kusz}suhub2-sze3          suhub           boots   
11   o.5.1   kux(KWU147)-ra-ni-sa6       Kuranisa               1   
12   r.1.1                     szu            szu            hand   
13   r.1.2                   ba-ti

/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_72273/4058718997.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_72273/4058718997.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_72273/4058718997.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_72273/4058718997.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

              id           form        lem eng_translation                tags
0     o.col1.1.1           1(u)       1(u)             ten                [NU]
1     o.col1.1.2         gurusz     gurusz            male                 [N]
2     o.col1.1.3          engar      engar          farmer                 [N]
3     o.col1.1.4        dumu-ni       dumu           child    [N, 3-SG-H-POSS]
4     o.col1.2.1        6(disz)    6(disz)             one                [NU]
..           ...            ...        ...             ...                 ...
374  r.col5.2'.2  nu-banda3-gu4  nubandagu     ox_overseer  [N, GEN, GEN, GEN]
375  r.col5.3'.1             mu         mu            year                 [N]
376  r.col5.3'.2         us2-sa         us          follow         [NF, V, PT]
377  r.col5.3'.3    ki-masz{ki}     Kimasz               1           [SN, ABS]
378  r.col5.3'.4         ba-hul    ba-hulu         destroy    [MID, V, 3-SG-S]

[379 rows x 5 columns]
      id               form 

/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_72273/4058718997.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_72273/4058718997.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_72273/4058718997.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_72273/4058718997.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

In [50]:
extracted_df

,id,form,lem,eng_translation,tags
0,o.1.1,1(disz),1(disz),one,[NU]
1,o.1.2,udu-nita2,udunita,ram,"[N, ABS]"
2,o.1.3,ba-usz2,ba-usz,die,"[MID, V, 3-SG-S]"
3,o.2.1,1(disz),1(disz),one,[NU]
4,o.2.2,sila4,sila,lamb,[N]
...,...,...,...,...,...
27,s1.col1.4.3,limmu2-ba,limmu,four,"[NU, 3-SG-NH-POSS, GEN, ABS]"
28,s1.col2.1.1,a-a-kal-la,Ayakala,1,[PN]
29,s1.col2.2.1,ensi2,ensik,ruler,[N]
30,s1.col2.3.1,umma{ki},Umma,1,"[SN, GEN, ABS]"


# unique tags

In [66]:
uniqe_tags = set()
for tg in extracted_df['tags']:
    for t in tg:
        if t not in ['','_']:
            uniqe_tags.add(t)
uniqe_tags = sorted(uniqe_tags)
uniqe_tags

['1-SG-POSS',
 '2-SG-A',
 '2-SG-POSS',
 '3-NH',
 '3-PL',
 '3-PL-DAT',
 '3-PL-S',
 '3-SG-A',
 '3-SG-DAT',
 '3-SG-H',
 '3-SG-H-A',
 '3-SG-H-DAT',
 '3-SG-H-POSS',
 '3-SG-HN-P',
 '3-SG-NH',
 '3-SG-NH-A',
 '3-SG-NH-L3',
 '3-SG-NH-P',
 '3-SG-NH-POSS',
 '3-SG-P',
 '3-SG-S',
 'ABL',
 'ABS',
 'AN',
 'ANT',
 'CNJ',
 'COM',
 'COP-3-PL',
 'COP-3-SG',
 'DAT',
 'DAT-H',
 'DAT-NH',
 'DEM2',
 'DN',
 'EN',
 'ERG',
 'F',
 'FIN',
 'FN',
 'GEN',
 'GEN-ABL',
 'GN',
 'L1',
 'L1-SYN',
 'L2',
 'L2-NH',
 'L3',
 'L3-NH',
 'MID',
 'MN',
 'MOD',
 'N',
 'NEG',
 'NF',
 'NU',
 'ON',
 'PF',
 'PL',
 'PN',
 'PT',
 'RDP',
 'RN',
 'SN',
 'SUB',
 'TERM',
 'TN',
 'V',
 'VEN',
 'WN']

In [67]:
len(uniqe_tags)

69

## mappings

In [74]:
# read json
filepath = '../Data/CDLI-CoNLL-to-CoNLLU-Converter/cdliconll2conllu/mapping.json'
cdliconll_map_data = json_file_read(filepath)

In [75]:
cdliconll_map_data

{'CDLI-CoNLL-Fields': ['ID',
  'FORM',
  'SEGM',
  'XPOSTAG',
  'HEAD',
  'DEPREL',
  'MISC'],
 'CoNLL-U-Fields': ['ID',
  'FORM',
  'LEMMA',
  'UPOSTAG',
  'XPOSTAG',
  'FEATS',
  'HEAD',
  'DEPREL',
  'DEPS',
  'MISC'],
 'xPOSTagMapping': {'AJ': 'ADJ',
  'AV': 'ADV',
  'NU': 'NUM',
  'CNJ': 'CCONJ',
  'DET': 'DET',
  'J': 'INTJ',
  'N': 'NOUN',
  'V': 'VERB',
  'PRP': 'ADP',
  'DN': 'PROPN',
  'EN': 'PROPN',
  'GN': 'PROPN',
  'MN': 'PROPN',
  'PN': 'PROPN',
  'RN': 'PROPN',
  'SN': 'PROPN',
  'TN': 'PROPN',
  'WN': 'PROPN',
  'AN': 'PROPN',
  'CN': 'PROPN',
  'FN': 'PROPN',
  'LN': 'PROPN',
  'ON': 'PROPN',
  'QN': 'PROPN',
  'YN': 'PROPN'},
 'posToFeatMapping': {'NOUN': ['Animacy', 'Case', 'Number'],
  'VERB': ['Aspect',
   'Mood',
   'Number',
   'Person',
   'Polarity',
   'Verb Form',
   'Voice'],
  'ADJ': ['Case', 'Number'],
  'NUM': ['Case'],
  'PROPN': ['Case', 'Number', 'Animacy']},
 'featureMapping': {'Case': {'ABS': 'Abs',
   'GEN': 'Gen',
   'ERG': 'Erg',
   'ABL': 'Abl'}

## Alphabets

In [60]:
alpha = set()
for word in extracted_df['form']:
    for char in word:
        alpha.add(char)
alpha = sorted(alpha)
print(len(alpha))
alphs_string = "".join(alpha)
alphs_string

66


"#'()-./0123456789:<>?@ABDEGIKLMNPRSTUWZ[]abcdeghiklmnpqrstuvwxz{|}"

### Add alphabets to dict

In [106]:
base_dix_path = './apertium-sux.sux.dix'

In [107]:
mytree = ET.parse(base_dix_path)

In [108]:
mytree = ET.parse(base_dix_path)
myroot = mytree.getroot()

In [109]:
root = mytree.getroot()
root

<Element 'dictionary' at 0x1241c45e0>

In [110]:
old_alpha = root.find('alphabet')
root.remove(old_alpha)

In [111]:
el_alpha = ET.Element("alphabet")
el_alpha.text = alphs_string

In [112]:
root.append(el_alpha)

In [113]:
tree = ET.ElementTree(root)
# ET.indent(tree, space="\t", level=0)

In [114]:
sux_dix_name = "apertium-sux.sux.dix.V1"
with open (sux_dix_name, "wb") as f :
        tree.write(f,encoding = "UTF-8", xml_declaration = True)

## Dictionary

In [64]:
sum_eng_dict_v1 = {}
for sux_lem,eng_trans in zip(extracted_df['lem'],extracted_df['eng_translation']):
    sum_eng_dict_v1[sux_lem] = eng_trans
print(len(sum_eng_dict_v1))
sum_eng_dict_v1

1590


{'1(disz)': 'one',
 'udunita': 'ram',
 'ba-usz': 'die',
 'sila': 'unit',
 'azur': 'type_of_sheep',
 'ki': 'place',
 'Uru': '1',
 'szag': 'heart',
 'bala': 'turn',
 'kiszib': 'seal',
 'Arad': '1',
 'iti': 'month',
 'Szesagku': '1',
 'mu': 'year',
 'us': 'follow',
 'Kimasz': '1',
 'ba-hulu': 'destroy',
 'Aradmu': '1',
 'dubsar': 'scribe',
 'dumu': 'child',
 'Urnigar': '1',
 'kusz': 'leather',
 'udu': 'sheep',
 'u': 'grass',
 'maszgal': 'billy_goat',
 'ba-ug': 'die',
 'ud': 'day',
 '1(u)': 'ten',
 'la': 'hang',
 't)': 'one',
 'Tummal': '1',
 'Endingirmu': '1',
 'irimu': '1',
 'szu': 'hand',
 '-teg': 'accept',
 'Akiti': '1',
 'en': 'priest',
 'Nanna': '1',
 'ba-hug': 'hire',
 '3(disz)': 'one',
 'gig': 'unit',
 '2(u)': 'ten',
 'sze': 'barley',
 'igi6(disz)': 'one',
 'Urbagara': '1',
 '1(ban)': 'unit',
 'inun': 'ghee',
 '6(disz)': 'one',
 'ga': 'milk',
 'gazi': 'condiment',
 'Urmes': '1',
 'Urdumuzi': '1',
 'Szugargal': '1',
 'niga': 'fattened',
 'Annahilibi': '1',
 'Lukala': '1',
 'e': '1',

# EXTRA

In [161]:
s = '1(disz)[one]'

In [180]:
re.findall(r"[\(\)a-zA-Z0-9_-]+\[\w+\]",s)

['1(disz)[one]']

In [167]:
# w_group = re.findall(r"[a-zA-Z0-9_-]+\[\w+\]",s)
w_group = re.findall(r"\(\.+\)",s)
print(w_group)
for w in w_group:
    start_index = w.find('[')
    end_index = w.find(']')
    lem = w[:start_index]
    eng_translation = w[start_index+1:end_index]
    print(lem,eng_translation)

[]
